In [1]:
import scanpy as sc
import numpy as np
import muon

from sklearn.model_selection import KFold

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

# Download data

In [2]:
# !wget --no-check-certificate https://covid19.cog.sanger.ac.uk/submissions/release1/haniffa21.processed.h5ad -O ../../data/raw/Haniffa_all.h5ad

In [3]:
adata = sc.read('../../data/raw/Haniffa_all.h5ad')
adata

AnnData object with n_obs × n_vars = 647366 × 24929
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [4]:
adt = adata[:, adata.var['feature_types'] == 'Antibody Capture'].copy()
adt

AnnData object with n_obs × n_vars = 647366 × 192
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [5]:
rna = adata[:, adata.var['feature_types'] == 'Gene Expression'].copy()
rna

AnnData object with n_obs × n_vars = 647366 × 24737
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'raw'

In [6]:
del adata

# Preprocess RNA

In [7]:
rna.layers['raw'].data

array([1., 1., 1., ..., 7., 3., 7.], dtype=float32)

In [8]:
rna.layers['counts'] = rna.layers['raw'].copy()
rna.X = rna.layers['counts'].copy()
del rna.layers['raw']

We include `Site` as the batch covariate in the hvg selection and compute PCAs, neighbors and UMAP coordinates using the hvgs.

In [9]:
sc.experimental.pp.highly_variable_genes(rna, n_top_genes=2000, batch_key='Site')
rna

AnnData object with n_obs × n_vars = 647366 × 24737
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types', 'means', 'variances', 'residual_variances', 'highly_variable_rank', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'counts'

In [10]:
rna = rna[:, rna.var.highly_variable].copy()
rna

AnnData object with n_obs × n_vars = 647366 × 2000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id'
    var: 'feature_types', 'means', 'variances', 'residual_variances', 'highly_variable_rank', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'counts'

# Preprocess ADT

In [11]:
adt.layers['raw'].data

array([ 2.,  6.,  2., ..., 48., 56., 48.], dtype=float32)

In [12]:
adt.layers['counts'] = adt.layers['raw'].copy()
adt.X = adt.layers['counts'].copy()
del adt.layers['raw']

In [13]:
muon.prot.pp.clr(adt)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/muon/_prot/preproc.py:219: UserWarning: adata.X is sparse but not in CSC format. Converting to CSC.
  warn("adata.X is sparse but not in CSC format. Converting to CSC.")


# Create 5-fold CV (based on patients)

In [14]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)

In [15]:
patients = np.unique(rna.obs['patient_id'])

In [16]:
train_test_sizes = []

for i, (train_index, test_index) in enumerate(kf.split(patients)):
    train_patients = patients[train_index]
    test_patients = patients[test_index]
    rna.obs.loc[rna.obs['patient_id'].isin(train_patients), f'split{i}'] = 'train'
    rna.obs.loc[~rna.obs['patient_id'].isin(train_patients), f'split{i}'] = 'val'
    adt.obs[f'split{i}'] = rna.obs[f'split{i}']
    
    len_rna_train = len(rna[rna.obs[f'split{i}'] == 'train'])
    len_rna_test = len(rna[rna.obs[f'split{i}'] == 'val'])
    train_conditions = set(rna[rna.obs[f'split{i}'] == 'train'].obs['Status_on_day_collection_summary'].cat.categories)
    test_conditions = set(rna[rna.obs[f'split{i}'] == 'val'].obs['Status_on_day_collection_summary'].cat.categories)
    assert test_conditions.issubset(train_conditions)
    train_test_sizes.append((len_rna_train, len_rna_test))

In [17]:
train_test_sizes

[(531289, 116077),
 (495537, 151829),
 (528595, 118771),
 (503281, 144085),
 (530762, 116604)]

In [18]:
rna.write('../../data/pbmc_full_rna.h5ad')
adt.write('../../data/pbmc_full_adt.h5ad')

# Adjust conditions

In [19]:
rna.obs['Status_on_day_collection_summary'].cat.categories

Index(['Asymptomatic', 'Critical', 'Healthy', 'LPS_10hours', 'LPS_90mins',
       'Mild', 'Moderate', 'Non_covid', 'Severe'],
      dtype='object')

In [20]:
drop_conditions = ['LPS_10hours', 'LPS_90mins', 'Non_covid']

In [21]:
rna = rna[~rna.obs['Status_on_day_collection_summary'].isin(drop_conditions)]
adt = adt[~adt.obs['Status_on_day_collection_summary'].isin(drop_conditions)]
(rna.shape, adt.shape)

((624325, 2000), (624325, 192))

In [22]:
rna.write('../../data/pbmc_healthy_covid_rna.h5ad')
adt.write('../../data/pbmc_healthy_covid_adt.h5ad')